In [ ]:
import requests
import json
import time
import pandas as pd
from IPython.display import Markdown  

In [ ]:
import os 
import ast

def read_list_from_env_file(file_path):# 从文件中读取列表
    with open(file_path) as f:
        content = f.read().strip()
    # 从字符串中解析出列表
    lst = ast.literal_eval(content)
    return lst

In [ ]:
api_keys=read_list_from_env_file(".env")# 读取.env文件中的API_KEY

In [ ]:
class APIPool:
    def __init__(self,tokens):
        self.tokens=tokens
        self.last_called={token:0 for token in tokens}  #上次调用时间
    def getapi(self):
        now = int(time.time())
        for token in self.tokens:
            if self.last_called[token]+21 <=now:    
                self.last_called[token]=now
                return token
        time.sleep(self.last_called[self.tokens[0]]+21-now) #等待
        return self.getapi() #重新调用
    def reset(self):
        self.last_called={token:0 for token in self.tokens}#重置

In [ ]:
print("hello world!")

In [ ]:
A=APIPool(api_keys) #创建API池

In [ ]:
url = "https://api.lixining.com/v1/chat/completions" # 选择一个API
#反向代理 如失效需要 更换到上面
#https://openai.1rmb.tk/v1/chat/completions
#https://ai.fakeopen.com/v1/chat/completions
#https://api.lixining.com/v1/chat/completions

In [ ]:
def get_header():# 获取请求头
    header={
  'Authorization': f'Bearer {A.getapi()}',
  'Content-Type': 'application/json'
    }
    return header

In [ ]:
print(get_header())

In [ ]:
#获取消息的函数
def get_completion(prompt):
    payload = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "system", "content": f"You are ChatGPT, a large language model trained by OpenAI. \
            Follow the user's instructions carefully. Respond using markdown."},
        {
          "role": "user",
          "content": f"{prompt}"
        }
      ]
    }
    data=None
    try:
        response = requests.post(url, headers=get_header(), json=payload)
        response.raise_for_status() # 抛出异常，如果响应码不是200
        data = response.json()
        return (data["choices"][0]["message"]["content"])
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e} \n {data}")
    except json.JSONDecodeError as e:
        print(f"无效的 JSON 响应: {e}\n {data}")
    except KeyError as e:
        print(f"解析 JSON 错误: {e}\n {data}")

In [ ]:
class TextStorage:
    def __init__(self):
        self.text = ""

    def add_text(self, new_text):
        self.text += str(new_text)

    def get_text(self):
        return self.text
    
    def clear_text(self):
        self.text = ""

In [ ]:
with open("./data/readinglist.json", "r") as f:
    reading_list = json.load(f)

In [ ]:
reading_list,type(reading_list)
T=TextStorage()

In [125]:
prompt = f"""
下面是三个引号括起来的JSON格式数据，增加一个列名为简介的列，把简介补充好,转换成HTML表格,表格要有分割线样式，只回答我html代码:

"""
for book in reading_list: 
    display((json.dumps(book["书名"],ensure_ascii=False)))
    response = get_completion(f"{prompt}+\"\"\"{json.dumps(book)}\"\"\"")
    T.add_text(response)
    

'"人月神话"'

'"操作系统概念"'

'"编译原理"'

'"Unix 编程艺术"'

解析 JSON 错误: 'choices'
 {'error': {'message': 'Rate limit reached for default-gpt-3.5-turbo in organization org-SjJboAKeF2Eoty5Z9L91NDDX on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.', 'type': 'requests', 'param': None, 'code': None}}


...

'"代码大全"'

'"人工智能：一种现代的方法"'

'"Head First 设计模式"'

'"人类简史：从动物到上帝"'

'"深度学习"'

In [ ]:
HTML(T.get_text())